# Test 2: Time Series Regression

## Air Quality Forecasting

**Author:** Virginia Herrero [Email](mailto:v.herrero@outlook.com) | [LinkedIn](https://www.linkedin.com/in/virginia-herrero-casero/) | [GitHub](https://github.com/herrerovir)

### Introducción

En este proyecto, se lleva a cabo un proceso de machine learning de principio a fin, con el objetivo de predecir la calidad del aire usando datos de series temporales. Este cuaderno detalla paso a paso todas las decisiones tomadas desde la limpieza de los datos hasta el desarrollo y evaluación del modelo.

Elegí este conjunto de datos porque es relevante para mi formación como ingeniero químico especializada en control de calidad. Predecir la calidad del aire es una gran oportunidad para mostrar mis habilidades en ingeniería aplicada a la ciencia de datos, especialmente en el ámbito del control de calidad para diferentes industrias.

### Dataset

El conjunto de datos utilizado en este proyecto se obtuvo del UCI Machine Learning Repository [here](https://archive.ics.uci.edu/dataset/360/air+quality).

La información proporcionada del conjunto de datos es la siguiente:

- The dataset contains 9358 instances of hourly averaged responses from an array of 5 metal oxide chemical sensors embedded in an Air Quality Chemical Multisensor Device. The device was located on the field in a significantly polluted area, at road level, within an Italian city. 

- Data were recorded from March 2004 to February 2005 (one year) representing the longest freely available recordings of on field deployed air quality chemical sensor devices responses. 

- Ground Truth hourly averaged concentrations for CO, Non Metanic Hydrocarbons, Benzene, Total Nitrogen Oxides (NOx) and Nitrogen Dioxide (NO2) and were provided by a co-located reference certified analyzer.

- Missing values are tagged with -200 value.

Las variables de este conjunto de datos son las siguientes:

- `Date`: (DD/MM/YYYY)

- `Time`: (HH.MM.SS)

- `CO(GT)`: True hourly averaged concentration CO in mg/m^3 (reference analyzer)

- `PT08.S1(CO)`: (tin oxide) hourly averaged sensor response (nominally CO targeted)

- `NMHC(GT)`: True hourly averaged overall Non Metanic HydroCarbons concentration in microg/m^3 (reference analyzer)

- `C6H6(GT)`: True hourly averaged Benzene concentration  in microg/m^3 (reference analyzer)

- `PT08.S2(NMHC)`: (titania) hourly averaged sensor response (nominally NMHC targeted)

- `NOx(GT)`: True hourly averaged NOx concentration  in ppb (reference analyzer)

- `PT08.S3(NOx)`: (tungsten oxide) hourly averaged sensor response (nominally NOx targeted)

- `NO2(GT)`: True hourly averaged NO2 concentration in microg/m^3 (reference analyzer)

- `PT08.S4(NO2)`: (tungsten oxide) hourly averaged sensor response (nominally NO2 targeted)

- `PT08.S5(O3)`: (indium oxide) hourly averaged sensor response (nominally O3 targeted)

- `T`: Temperature in °C

- `RH`: Relative Humidity (%)

- `AH`: Absolute Humidity

### Data loading

Carga el archivo CSV **AirQualityUCI** como un DataFrame de pandas.

In [ ]:
# Import all required libraries
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Seasonal analysis
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

# Modeling and evaluation
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Load the dataset
df = pd.read_csv("AirQualityUCI.csv", delimiter = ";", decimal = ",")
df.head()

### Data cleaning

Limpia y preprocesa el conjunto de datos antes de seguir con el análisis.

In [ ]:
df.info()

* **Elimina columnas innecesarias**

Las columnas Unnamed: 15 y Unnamed: 16 están vacías, por lo tanto, se eliminan del conjunto de datos.

In [ ]:
df = df.drop(["Unnamed: 15", "Unnamed: 16"], axis = 1)

* **Renombra columnas**

Algunas columnas se renombraron para proporcionar un contexto más claro y mejorar la comprensión para una audiencia no técnica del conjunto de datos.

In [ ]:
df.columns

In [ ]:
df = df.rename(columns = {"CO(GT)" : "CO_Concentration",
                          "PT08.S1(CO)" : "CO_Sensor_Response",
                          "NMHC(GT)" : "NMHC_Concentration",
                          "C6H6(GT)" : "C6H6_Concentration",
                          "PT08.S2(NMHC)" : "NMHC_Sensor_Response",
                          "NOx(GT)" : "NOx_Concentration",
                          "PT08.S3(NOx)" : "NOx_Sensor_Response",
                          "NO2(GT)" : "NO2_Concentration",
                          "PT08.S4(NO2)" : "NO2_Sensor_Response",
                          "PT08.S5(O3)" : "O3_Sensor_Response",
                          "T" : "Temperature",
                          "RH" : "Relative_Humidity",
                          "AH" : "Absolute_Humidity"})

In [ ]:
# Show the dataset with the renamed variables
df.head()

* **Valores nulos**

Identifica y elimina cualquier valor nulo en el conjunto de datos cuando sea necesario.

In [ ]:
# Check the total of null values in each column
df.isna().sum()

Hay 114 valores nules en todas las columnas del dataset.

In [ ]:
# Show all the rows with missing values
missing_rows = df[df.isna().any(axis = 1)]
missing_rows

El conjunto de datos tiene un total de 114 valores faltantes, lo cual es un porcentaje relativamente bajo, aproximadamente el 1.2% de todo el conjunto de datos. Dado este pequeño porcentaje, eliminaré todas las filas con valores nulos. Dado que se trata de una serie temporal, utilizar un método de imputación como el fill forward podría crear valores duplicados, ya que simplemente llevaría hacia adelante la última entrada no NaN disponible. Por lo tanto, eliminar estas filas es el mejor método para mantener la integridad de los datos.

In [ ]:
# Drop missing values
df = df.dropna()

La descripción del conjunto de datos indica que los valores faltantes están etiquetados con el valor -200. Por lo tanto, se buscarán todos los valores -200 y se reemplazarán por valores NaN.

In [ ]:
# Count -200 values in each column
df.apply(lambda x : x == -200).sum()

El análisis del conteo de valores -200 muestra claramente que la característica **"NMHC_Concentration"** tiene una cantidad significativa de valores nulos. Por esta razón, esta columna se eliminará directamente del conjunto de datos. Posteriormente, todas las instancias de -200 se reemplazarán por valores NaN y se imputarán siguiendo el método de relleno hacia adelante, el cual es adecuado para las series temporales.

In [ ]:
# Drop the colum with too many missing values
df.drop(columns = ["NMHC_Concentration"], inplace = True)

In [ ]:
# Replace values with NaN
df.replace(to_replace = -200, value = np.nan, inplace = True)

In [ ]:
# Forward fill missing values
df.ffill(inplace = True)

In [ ]:
# Check remaining missing values
df.isna().sum()

* **Valores duplicados**

Verifica si hay entradas duplicadas en el conjunto de datos.

In [ ]:
df.duplicated().sum()

El dataset no tiene valores duplicados.

* **Tipos de datos**

Verifica que todas las columnas tengan los tipos de datos apropiados.

In [ ]:
df.dtypes

Las columnas **"Date"** y **"Time"** están configuradas como tipos objeto, por lo que necesitan ser convertidas al tipo datetime.

In [ ]:
# Create new feature with date and time
df["Datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"], format = "%d/%m/%Y %H.%M.%S")

# Set the new feature as the index
df = df.set_index("Datetime")

# Drop the unnecesary columns
df = df.drop(["Date", "Time"], axis = 1)
df

In [ ]:
# Check again for missing values after transformation
df.isna().sum()

* **Outliers**

Examina el resumen estadístico del conjunto de datos para identificar posibles valores atípicos. Esta visión general inicial ayudará a resaltar cualquier valor inusual que pueda requerir una exploración adicional.

In [ ]:
df.describe().T

A partir de las estadísticas resumidas del conjunto de datos, es evidente que las siguientes variables muestran discrepancias importantes entre sus valores máximos y el cuartil superior, lo que indica la existencia de posibles valores atípicos.

- CO_Concentration
- C6H6_Concentration
- NOx_Concentration

Primero, se trazarán graficas de estas variables para identificar posibles valores atípicos. Luego, los valores atípicos se identificarán utilizando el método del rango intercuartílico (IQR). El análisis de la distribución de los datos revelará los valores que están fuera del rango típico, lo que permitirá seleccionar un enfoque adecuado para manejarlos.

In [ ]:
# Plot the CO concentration distribution using a histogram and boxplot
CO_concentration_distribution = plt.figure()
fig, ax = plt.subplots(1, 2, figsize = (11, 3))
sns.histplot(df["CO_Concentration"], ax = ax[0], color = "#e7298a")
sns.boxplot(x = df["CO_Concentration"], ax = ax[1], color = "#e7298a")
ax[0].set_xlabel("Concentración de CO")
ax[1].set_xlabel("Concentración de CO")
plt.suptitle("Distribución de la concentración de CO", size = 12)

In [ ]:
# Plot the C6H6 concentration distribution using a histogram and boxplot
C6H6_concentration_distribution = plt.figure()
fig, ax = plt.subplots(1, 2, figsize = (11, 3))
sns.histplot(df["C6H6_Concentration"], ax = ax[0], color = "#e7298a")
sns.boxplot(x = df["C6H6_Concentration"], ax = ax[1], color = "#e7298a")
ax[0].set_xlabel("Concentración de C$_6$H$_6$")
ax[1].set_xlabel("Concentración de C$_6$H$_6$")
plt.suptitle("Distribución de la concentración de C$_6$H$_6$", size = 12)

In [ ]:
# Plot the NOx concentration distribution using a histogram and boxplot
NOx_concentration_distribution = plt.figure()
fig, ax = plt.subplots(1, 2, figsize = (11, 3))
sns.histplot(df["NOx_Concentration"], ax = ax[0], color = "#e7298a")
sns.boxplot(x = df["NOx_Concentration"], ax = ax[1], color = "#e7298a")
ax[0].set_xlabel("Concentración de NO$_x$")
ax[1].set_xlabel("Concentración de NO$_x$")
plt.suptitle("Distribución de la concentración de NO$_x$", size = 12)

Encuentra los valores atípicos utilizando el método del rango intercuartílico (IQR).

In [ ]:
# Create a function to find outliers using the IQR method

def find_outliers_iqr(dataframe, column):
    """
    Finds outliers in the specified column of a DataFrame using the IQR method

    Parameters
    ----------
    dataframe : Pandas DataFrame
        The DataFrame containing the data
    
    column : str
        The name of the column (as a string) in which to find the outliers

    Returns
    -------
    Pandas DataFrame
        A DataFrame containing the outliers identified in the specified column
   """
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)
    
    # Calculate IQR
    IQR = Q3 - Q1
    
    # Determine the bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers = dataframe[(dataframe[column] < lower_bound) | (dataframe[column] > upper_bound)]
    print(f"The number of outliers in the column {column} is {outliers.shape[0]}")
    
    return outliers

In [ ]:
# Find outliers in the dataset
for i in df.columns:
    find_outliers_iqr(df, i)

A partir del análisis estadístico de los valores atípicos, se puede concluir que hay una gran cantidad de valores fuera de rango. Teniendo en cuenta la naturaleza de las características, los valores atípicos podrían estar presentes debido a picos de contaminación en el aire. Por lo tanto, estos datos podrían ser relevantes. Sin embargo, la presencia de valores atípicos puede afectar el rendimiento del modelo que se utilizarán posteriormente para predecir los 100 ciclos. Por estas razones, he decidido limitar los valores atípicos para conservar todos los datos, pero reducir el posible impacto negativo que puedan tener en el modelo predictivo.

In [ ]:
def cap_outliers(dataframe, column):
    """
    Cap outliers in a specified column of the DataFrame using the IQR method

    Parameters
    ----------
    dataframe : Pandas DataFrame
        The DataFrame containing the data
    
    column : str
        The name of the column to cap outliers


    Returns
    -------
    Pandas DataFrame
        A DataFrame with outliers capped
    """
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    
    # Calculate Q1 and Q3
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)

    # Calculate IQR

    IQR = Q3 - Q1

    # Define bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Cap the outliers
    dataframe[column] = dataframe[column].clip(lower = lower_bound, upper = upper_bound)

    return dataframe

In [ ]:
# Cap outliers
for i in df.columns:
    cap_outliers(df, i)

La distribución de las características después de limitar los outliers es la siguiente: 

In [ ]:
df.hist(figsize = (20, 10), color = "#e7298a")
plt.suptitle("Distribución de las características")

* **El conjunto de datos limpio:**

In [ ]:
air_quality = df.copy()
air_quality.head()

In [ ]:
# Save the cleaned dataset
air_quality.to_csv("Air-quality-cleaned.csv")

### Seasonality

El requerimiento de este ejercicio es que la serie temporal sea **no-estacional**. La estacionalidad se refiere a las fluctuaciones periódicas de un conjunto de datos. Estos intervalos regulares se producen debido a factores estacionales. Para determinar si la serie temporal es no-estacional, primero hay que comprobar si la serie es **estacionaria**. Para ello se emplea la prueba estadística de Dickey-Fuller la cual consiste en detectar la presencia de una raíz unitaria la cual indica no estacionariedad. Si el resultado de la prueba es significativo, sugiere que la serie es estacionaria, y por tanto, no presenta estacionalidad.

In [ ]:
def check_seasonality(series):
    """
    Perform the Dickey-Fuller test to check for seasonality and print the results.

    Parameters
    ----------
    series : Pandas Series
        The time series data to test for seasonality.
    
    Returns
    -------
    None
    """
    result = adfuller(series)
    adf_statistic = result[0]
    p_value = result[1]
    critical_values = result[4]
    
    # Print the results
    print("Dickey-Fuller Test Results:\n")
    print(f"ADF Statistic: {adf_statistic}")
    print(f"p-value: {p_value}")

    for key, value in critical_values.items():
        print(f"Critical Value {key}: {value}")
    
    # Interpretation
    interpretation = ("The time series is stationary, indicating the absence of seasonality."
                      if p_value < 0.05 else
                      "The time series is non-stationary, suggesting it may  seasonality.")
    print(f"\nInterpretation: {interpretation}")


In [ ]:
# Check seasonality on target variable
adf_result = check_seasonality(air_quality["NO2_Concentration"])

Los resultados de la prueba de Dickey-Fuller indican que la serie temporal es **no estacional**.

- **Estacionariedad**: La estadística ADF es menor que los valores críticos y el valor p es muy bajo. Esto significa que la serie es estacionaria, lo que sugiere que no tiene tendencias ni fluctuaciones que cambien con el tiempo.

- **No Estacionalidad**: La falta de estacionalidad significa que no hay patrones repetidos en intervalos regulares. Esto implica que la serie no muestra variaciones predecibles según la época del año o del mes.

Por estas razones, podemos afirmar que la serie temporal es no estacional y estacionaria.

In [ ]:
# Plot the NO2 concentration over time
plt.figure(figsize = (11, 4))
plt.plot(air_quality["NO2_Concentration"], color = "#e7298a")
plt.xlabel("Fecha")
plt.ylabel("Concentración de NO$_2$")
plt.title("Concentración de NO$_2$ a lo largo del tiempo")

In [ ]:
# Perform seasonal decomposition of the NO2 concentration data
result = seasonal_decompose(air_quality["NO2_Concentration"], model = "additive", period = 365)

# Plot the decomposition results
plt.figure(figsize = (14, 10))
result.plot()
plt.suptitle("Descomposición Estacional de la Concentración de NO$_2$")

# Change the color of the lines in the plot and set axis labels
for ax in plt.gcf().axes:
    for line in ax.lines:
        line.set_color("#e7298a")
    ax.set_xlabel("Tiempo")

plt.tight_layout()

Los resultados de la descomposición estacional muestran los componentes de la serie temporal para comprender mejor la dinámica de la concentración de NO<sub>2</sub> a lo largo del tiempo:

1. **Observado**: datos originales de las concentraciones de NO<sub>2</sub>
2. **Tendencia**: tendencia general a largo plazo de las concentraciones de NO<sub>2</sub>
3. **Estacional**: patrones repetitivos en intervalos regulares de las concentraciones de NO<sub>2</sub>
4. **Residual**: fluctuaciones aleatorias no explicadas por la tendencia o estacionalidad

### Exploratory Data Analysis

#### **Target Variable**

La variable objetivo de este proyecto es la concentración de NO<sub>2</sub>. He elegido esta variable por las siguientes razones:

- De entre todos los contaminantes del dataset, NO<sub>2</sub> es uno de los más perjudiciales para la salud.
- Los efectos de NO<sub>2</sub> sobre la salud están bien documentados.
- La regulación establece valores máximos y claros para este gas.

In [ ]:
air_quality.columns

- **Concentraciones mensuales de NO<sub>2</sub>**

In [ ]:
# Resample and plot the monthly mean of NO2 concentration levels
monthly_no2 = air_quality["NO2_Concentration"].resample("ME").mean()

ax = monthly_no2.plot(kind = "bar", figsize = (11, 4), color = "#e7298a")
plt.xlabel("Mes")
plt.ylabel("Concentración promedio de NO$_2$")
ax.set_xticks(range(len(monthly_no2)))
ax.set_xticklabels(monthly_no2.index.strftime("%B, %Y"))
plt.title("Concentraciones promedio mensuales de NO$_2$");       

Las concentraciones de NO<sub>2</sub> entre marzo de 2004 y abril de 2005 muestran que las concentraciones de este gas aumentan durante los meses de invierno y durante el verano decrecen hasta llegar al mínimo en agosto. Esto probablemente es debido al uso de calefacciónes en las casas y comercios, el transporte y mayor uso de electricidad. variaron. También se observa que existe una tendencia general al alza en los niveles de NO<sub>2</sub>.

- **Concentraciones de NO<sub>2</sub> por hora**

In [ ]:
# Resample the data to get hourly mean values
hourly_no2 = air_quality["NO2_Concentration"].resample("h").mean()

# Create a bar plot
plt.figure(figsize=(14, 6))
sns.barplot(x=hourly_no2.index.hour, y=hourly_no2.values, errorbar = None, color = "#e7298a")
plt.xlabel("Hora del día")
plt.ylabel("Concentración promedio de NO$_2$")
plt.title("Concentración promedio por hora de NO$_2$")
plt.xticks(range(24), [f"{h}:00" for h in range(24)]);

Los niveles de NO<sub>2</sub> fluctúan durante el día de manera significativa, oscilando entre 92.0 y 190.0 ppb. Las concentraciones comienzan en 113.0 ppb a medianoche y aumentan a lo largo de la tarde, alcanzando un pico de 122.0 ppb alrededor de las 21:00. Los niveles son más bajos durante la noche, probablemente debido a un menor uso del transporte, electricidad, etc.

#### **Features**

In [ ]:
# Drop unnecessary columns and resample the data by month, calculating the mean
monthly_mean = air_quality.drop(["Temperature", "Relative_Humidity", "Absolute_Humidity"], axis=1).resample("ME").mean()

In [ ]:
# Plot monthy average values of all the gases
colors = ["black", "blue", "deepskyblue", "lightseagreen", "limegreen", "gold", "darkorange", "chocolate", "deeppink"]

monthly_mean.plot(figsize=(15, 8), color=colors)
plt.xlabel("Mes")
plt.ylabel("Concentración promedio de gases tóxicos")
plt.title("Concentración mensual promedio de gases tóxicos")
plt.legend(loc="upper right")
plt.tight_layout()

Los gráfica que representa las concentraciones mensuales promedios de los gases en el aire, muestra que éstas fluctuan significativamente a lo largo del tiempo. La única que se mantiene bastante estable es la concentración de CO se mantiene bastante estable. Por otro lado, el benceno muestra más variabilidad. Las concentraciones de NO<sub>x</sub> y NO<sub>2</sub> son mucho más altas durante el otoño e invierno, lo que probablemente sea debido al aumento en el uso de calefacción y tráfico.

#### **Matriz de Correlación**

In [ ]:
correlations = air_quality.corr(numeric_only = True)
correlations

In [ ]:
correlation_heatmap_graph = plt.figure(figsize = (11, 4))
sns.heatmap(correlations, linewidths = 0.5, annot = True, cmap = "PuRd")
plt.title("Correlation Heatmap", size = 12)

La matriz de correlación muestra las relaciones entre las diferentes variables, cuánto más fuerte es el color, más relación hay entre las variables. Las relaciones más importantes entre los parámetros de calidad del aire son los siguientes:

1. **Correlaciones Fuertes**:
   - La concentración de CO se relaciona estrechamente con su sensor y con la concentración de C6H<sub>6</sub>.
   - Las respuestas de los sensores de C6H<sub>6</sub> y NMHC perfectamente correlacionadas.

2. **Compuestos de Nitrógeno**:
   - NOx y NO<sub>2</sub> tienen una fuerte correlación positiva, lo que significa que cuando uno aumenta, el otro también aumenta.
   - La respuesta del sensor de NOx se correlaciona negativamente con su concentración. Esto puede indicar que el sensor no funciona correctamente.

3. **Temperatura y Humedad**:
   - La temperatura tiene una leve correlación positiva con el sensor de NO<sub>2</sub> y negativa con CO.
   - La humedad absoluta se correlaciona más fuerte con la temperatura y el sensor de NO<sub>2</sub>.

## Pre-processing

- **Eliminación de Columnas**

Se eliminaron las columnas **NMHC_Sensor_Response** y **NOx_Sensor_Response** debido a:

- **NOx_Sensor_Response**: Fuerte correlación negativa con varias características y redundancia con NOx_Concentration.
  
- **NMHC**: Su alta correlación con C6H<sub>6</sub> sugiere que los niveles de ambos están midiendo compuestos similares y supone redundancia de datos.

Eliminando ambas características evitamos posibles problemas de multicolinealidad en el modelo predictivo.

In [ ]:
air_quality = air_quality.drop(["NMHC_Sensor_Response", "C6H6_Concentration"], axis=1)

In [ ]:
air_quality.head()

### LSTM Modeling

In [ ]:
# Create function to prepare the data for LSTM model
def create_dataset(data, time_step = 1):
    X, Y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step)])
        Y.append(data[i + time_step])
    return np.array(X), np.array(Y)

# Normalize the data
data = air_quality["NO2_Concentration"].values
data = (data - np.min(data)) / (np.max(data) - np.min(data))

# Set the time steps to create sequences for the model
time_step = 24

# Create the dataset to feed to the LSTM model
X, y = create_dataset(data, time_step)

# Reshape the data to be compatible with the LSTM model
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split the data into training and testing sets
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer = "adam", loss = "mean_squared_error")

# Train the model
early_stopping = EarlyStopping(monitor = "val_loss", patience = 5)
history = model.fit(X_train, y_train, epochs = 50, batch_size = 64, validation_split = 0.2, callbacks = [early_stopping])

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_test = y_test.reshape(-1, 1)
y_pred = y_pred.reshape(-1, 1)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print results
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")

Este modelo, con dos capas de LSTM y una completamente conectada es una excelente opción para resolver este problema por las siguientes razones:

- **Generalización**: la pérdida de validación se mantiene cercana a la pérdida de entrenamiento, lo que indica una buena capacidad de generalización a datos no vistos.
- **Métricas de Error**: las métricas de evaluación del modelo reflejan predicciones confiables.
- **Poder Predictivo**: el alto valor de R² demuestra que el modelo captura eficazmente las tendencias en los datos.

In [ ]:
# Save the model
model.save("LSTM-model-air-quality.keras")

**Pérdida de entrenamieto y validación**

In [ ]:
# Plot training and validation loss curves
plt.figure(figsize = (11, 4))
plt.plot(history.history["loss"], label = "Pérdida de entrenamiento", color = "#e7298a")
plt.plot(history.history["val_loss"], label = "Pérdida de validación", color = "deepskyblue")
plt.title("Pérdida del modelo LSTM a lo largo de las épocas")
plt.ylabel("Pérdida")
plt.xlabel("Época")
plt.legend()

**Predicciones vs Valores Reales y valor R2**

In [ ]:
# Calcular r2 score
r2 = r2_score(y_test, y_pred)

# Plot predictions vs actual values
plt.figure(figsize =  (11, 4))
plt.plot(y_test, label="Concentración de NO$_2$ Real", color = "#e7298a")
plt.plot(y_pred, label="Concentración de NO$_2$ Predicha", color = "deepskyblue")
plt.title(f"Predicciones vs Valores Reales (R² = {r2:.2f})")
plt.ylabel("Concentración de NO$_2$")
plt.xlabel("Pasos de Tiempo")
plt.legend()

**Gráfica de residuos**

In [ ]:
# Calculate residual
residuals = y_test - y_pred

# Plot residuals
plt.figure(figsize=(11, 4))
plt.scatter(range(len(residuals)), residuals, color = "#e7298a")
plt.axhline(y = 0, color = "deepskyblue", linestyle = "--")
plt.title("Residuos")
plt.ylabel("Residuos")
plt.xlabel("Índice")


**Histograma de residuos**

In [ ]:
# Plot residuals histogram
plt.figure(figsize = (11, 4))
plt.hist(residuals, bins = 30, alpha = 0.7, color = "#e7298a")
plt.title("Histograma de residuos")
plt.xlabel("Residuos")
plt.ylabel("Frequencia")

### Forecasting

Otro de los requisitos de este ejercico es predecir 100 ciclos en el futuro sin utilizar las variables regresoras. Para ello se prepara la última entrada del dataset para que sea compatible con el modelo LSTM. Luego, se itera 100 veces para predecir los siguientes valores, agregando cada nuevo valor pronosticado a una lista que los guarda todos. Finalmente, se imprimen en la consola las predicciones para los próximos 100 pasos de tiempo.

In [ ]:
# Number of time steps to forecast
num_forecasts = 100

# Reshape input to fit LSTM
last_input = data[-time_step:].reshape(1, time_step, 1)

# List to store the forecasted values
forecasted_values = []

# Iterate to predict the next value
for _ in range(num_forecasts):
    next_value = model.predict(last_input) 
    forecasted_values.append(next_value[0, 0])
    last_input = np.concatenate((last_input[:, 1:, :], next_value.reshape(1, 1, 1)), axis=1)

# Convert the forecasted values to a numpy array
forecasted_values = np.array(forecasted_values)

# Print the forecasted values
print(f"Forecasted values for the next 100 time steps:\n {forecasted_values}")

### Conclusiones

El objetivo de este ejercicio fue crear un modelo de machine learning para predecir la calidad del aire en el futuro. El primer paso y uno de los más importantes es la limpieza de los datos. Este es una etapa esencial en el análisis de datos para asegurar que la información sea íntegra y confiable. Además, Se verificó que no hubiera estacionalidad en los datos, tal como se mencionaba en el planteamiento del problema. Antes del modelado se llevó a cabo una exploración de las variables para descubrir información relevante, tendencias y relaciones entre las variables. 

El modelo elegido para este proyecto fue una red neuronal recurrente, en concreto LSTM (Long Short-Term Memory). Lo interesante de la LSTM es que está hecha para aprender de datos en secuencia, lo que la hace perfecta para predecir la calidad del aire

Al final, se logró construir un modelo sólido que mostró métricas de evaluación muy buenas y una alta puntuación R², lo que indica un rendimiento efectivo. Por último, se realizaron predicciones para los próximos 100 ciclos sin utilizar variables de regresión, tal como se había solicitado en el ejercicio.